In [1]:
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pathlib import Path
import os
import re
import logging

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Configuração de logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Inicializa a sessão do Spark com configurações adicionais
spark = SparkSession.builder \
    .appName("HealthInsuranceIngestion") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
    .getOrCreate()

In [3]:
# Limpa o cache do Spark
spark.catalog.clearCache()

# Caminhos de entrada e saída
base_path = "/content/drive/My Drive/projetos/Projeto Integrador/dataset/raw/"
output_path = "/content/drive/My Drive/projetos/Projeto Integrador/dataset/bronze/"
os.makedirs(output_path, exist_ok=True)

In [4]:
def normalize_file_name(file_name):
    """Remove tudo após '_PUF' no nome do arquivo."""
    return re.sub(r"_PUF.*", "", file_name)

def process_and_save_file(file_path, table_name):
    """Lê um arquivo CSV, ajusta o schema dinamicamente e salva como Parquet com append."""
    try:
        # Lê o arquivo CSV
        df = spark.read.option("header", "true").csv(file_path)
        df = df.select([col(c).cast("string").alias(c) for c in df.columns])  # Converte todas as colunas para string

        print(f"Schema da tabela {table_name} antes da união:")
        df.printSchema()

        # Caminho da tabela no diretório de saída
        table_path = os.path.join(output_path, table_name)

        # Verifica se o diretório de saída já possui arquivos Parquet
        if Path(table_path).exists() and any(Path(table_path).glob("*.parquet")):
            existing_df = spark.read.parquet(table_path)
            existing_df = existing_df.select(
                [col(c).cast("string").alias(c) for c in existing_df.columns]
            )
            combined_df = existing_df.unionByName(df, allowMissingColumns=True)
        else:
            combined_df = df

        # Grava o DataFrame no diretório de saída
        combined_df.write.mode("append").parquet(table_path)
        print(f"Tabela {table_name} salva com sucesso.")
    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")
        print(f"Verifique o diretório de saída: {table_path}")

# Percorre todos os diretórios dentro de base_path
for year_dir in os.listdir(base_path):
    year_path = os.path.join(base_path, year_dir)
    if os.path.isdir(year_path):
        print(f"Arquivos no diretório {year_dir}:")
        for file_name in os.listdir(year_path):
            print(f" - {file_name}")
            if file_name.endswith(".csv"):
                normalized_name = normalize_file_name(file_name)
                table_name = f"tb_{normalized_name.lower()}"
                file_path = os.path.join(year_path, file_name)
                process_and_save_file(file_path, table_name)

print("Processamento concluído.")

Arquivos no diretório 2014:
 - .sentinel
 - Business_Rules_PUF.csv
Schema da tabela tb_business_rules antes da união:
root
 |-- BusinessYear: string (nullable = true)
 |-- StateCode: string (nullable = true)
 |-- IssuerId: string (nullable = true)
 |-- SourceName: string (nullable = true)
 |-- VersionNum: string (nullable = true)
 |-- ImportDate: string (nullable = true)
 |-- IssuerId2: string (nullable = true)
 |-- TIN: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- StandardComponentId: string (nullable = true)
 |-- EnrolleeContractRateDeterminationRule: string (nullable = true)
 |-- TwoParentFamilyMaxDependentsRule: string (nullable = true)
 |-- SingleParentFamilyMaxDependentsRule: string (nullable = true)
 |-- DependentMaximumAgRule: string (nullable = true)
 |-- ChildrenOnlyContractMaxChildrenRule: string (nullable = true)
 |-- DomesticPartnerAsSpouseIndicator: string (nullable = true)
 |-- SameSexPartnerAsSpouseIndicator: string (nullable = true)
 |-- AgeDe